In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# from datetime import datetime
import tensorflow as tf
import numpy as np
import random
import time
from tensorflow.examples.tutorials.mnist import input_data
from matplotlib import pyplot as plt
print(tf.test.is_gpu_available())

/home/praveenb/snapshot-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/praveenb/snapshot-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/praveenb/snapshot-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/praveenb/snapshot-env/lib/python3.6/site-packages/tensorflow/

True


In [2]:
batch_size = 64
learning_rate = 0.0002

n_epoch = 40
iterations_per_epoch = 300

## Model

In [3]:
def generator(features, training, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        conv1 = tf.nn.conv2d_transpose(features, 128, 4, stride=1, activation=tf.nn.relu)
        conv2 = tf.nn.conv2d_transpose(conv1, 1024, 4, stride=1, activation=tf.nn.relu)
        conv3 = tf.nn.conv2d_transpose(conv2, 512, 4, stride=2, activation=tf.nn.relu)
        conv4 = tf.nn.conv2d_transpose(conv3, 256, 4, stride=2, activation=tf.nn.relu)
        conv5 = tf.nn.conv2d_transpose(conv4, 128, 4, stride=2, activation=tf.nn.relu)
        conv6 = tf.nn.conv2d_transpose(conv5, stride=2, activation=tf.nn.relu)
        return conv6

In [4]:
def discriminator(features, training, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # 128 (4, 4), stride2+relu
        conv1 = tf.nn.conv2d(features, 128, 4, stride=2, activation=tf.nn.relu)
        # 256 // // //
        conv2 = tf.nn.conv2d(conv1, 256, 4, stride=2, activation=tf.nn.relu)
        # 512 // // //
        conv3 = tf.nn.conv2d(conv2, 512, 4, stride=2, activation=tf.nn.relu)
        # 1024 // // //
        conv4 = tf.nn.conv2d(conv3, 1024, 4, stride=2, activation=tf.nn.relu)
        # 1, //, stride 1 //
        conv5 = tf.nn.conv2d(conv4, 1, 4, stride=2, activation=tf.nn.relu)
        return conv5

## Placeholders

In [5]:
x = tf.placeholder(tf.float32, shape=(None, 64, 64, 1))
z = tf.placeholder(tf.float32, shape=(None, 1, 1, 150))
is_training = tf.placeholder(dtype=tf.bool)

In [6]:
g = generator(z, False)
d_real = discriminator(x, False)
d_fake = discriminator(g, True)

TypeError: conv2d_transpose() got an unexpected keyword argument 'stride'

In [ ]:
disc_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_real,
                                                                        labels=tf.ones([batch_size, 1, 1, 1])))
disc_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake,
                                                                        labels=tf.zeros([batch_size, 1, 1, 1])))
disc_loss_total = disc_loss_real + disc_loss_fake

In [ ]:
gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake,
                                                                  labels=tf.ones([None, 1, 1, 1])))

In [ ]:
t_vars = tf.trainable_variables()
disc_var = [var for var in t_vars if var.name.startswith('discriminator')]
gen_var = [var for var in t_vars if var.name.startswith('generator')]

In [ ]:
opt1 = tf.train.AdamOptimizer(learning_rate=learning_rate)
opt2 = tf.train.AdamOptimizer(learning_rate=learning_rate)
train1 = opt1.minimize(gen_loss)
train2 = opt2.minimize(disc_loss_total)